In [1]:
#!pip3 install smart_open[all]

In [2]:
import smart_open
import json

In [3]:
# def fork_event_pipeline(ctx, input_pattern, s3_output_path, sm = None):
#     """test pipeline to extract fork evenets across years"""
#     if sm is None:
#         sm = tuplex.dataset.SamplingMode.FIRST_FILE | tuplex.dataset.SamplingMode.LAST_FILE | tuplex.dataset.SamplingMode.FIRST_ROWS | tuplex.dataset.SamplingMode.LAST_ROWS
#     #sm = tuplex.dataset.SamplingMode.FIRST_FILE | tuplex.dataset.SamplingMode.FIRST_ROWS
#     #sm = tuplex.dataset.SamplingMode.FIRST_FILE | tuplex.dataset.SamplingMode.FIRST_ROWS | tuplex.dataset.SamplingMode.LAST_ROWS
#     #sm = tuplex.dataset.SamplingMode.LAST_FILE | tuplex.dataset.SamplingMode.FIRST_ROWS | tuplex.dataset.SamplingMode.LAST_ROWS

#     ctx.json(input_pattern, sampling_mode=sm) \
#         .withColumn('year', lambda x: int(x['created_at'].split('-')[0])) \
#         .withColumn('repo_id', extract_repo_id_code) \
#         .filter(lambda x: x['type'] == 'ForkEvent') \
#         .selectColumns(['type', 'repo_id', 'year']) \
#         .tocsv(s3_output_path)


In [4]:
from tqdm import tqdm

In [5]:
paths = ['s3://tuplex-public/data/github_daily/{}-10-15.json'.format(year) for year in range(2011, 2022)]
paths

['s3://tuplex-public/data/github_daily/2011-10-15.json',
 's3://tuplex-public/data/github_daily/2012-10-15.json',
 's3://tuplex-public/data/github_daily/2013-10-15.json',
 's3://tuplex-public/data/github_daily/2014-10-15.json',
 's3://tuplex-public/data/github_daily/2015-10-15.json',
 's3://tuplex-public/data/github_daily/2016-10-15.json',
 's3://tuplex-public/data/github_daily/2017-10-15.json',
 's3://tuplex-public/data/github_daily/2018-10-15.json',
 's3://tuplex-public/data/github_daily/2019-10-15.json',
 's3://tuplex-public/data/github_daily/2020-10-15.json',
 's3://tuplex-public/data/github_daily/2021-10-15.json']

In [20]:
# execute pipeline for a single line
def github_pipeline(line):
    rows = [json.loads(line)]
    
    def extract_repo_id(row):
        if 2012 <= row['year'] <= 2014:
            return row['repository']['id']
        else:
            return row['repo']['id']
        
    def with_column(rows, name, f):
        def helper(row):
            row[name] = f(row)
            return row
            
        return [helper(row) for row in rows]
        
    # now process
    tmp1 = with_column(rows, 'year', lambda x: int(x['created_at'].split('-')[0]))
    
    tmp2 = with_column(tmp1, 'repo_id', extract_repo_id)
    # how many exceptions and of what kind?
    tmp3 = list(filter(lambda x: not isinstance(x, Exception) and x['type'] == 'ForkEvent', tmp2))
    # get result
    tmp4 = list(map(lambda x: (x['type'], x['repo_id'], x['year']), tmp3))

    res = tmp4
    return res

In [24]:
rows = []
for s3_uri in paths:
    with smart_open.smart_open(s3_uri) as fin:
        row = {'path' : s3_uri}
        num_input_rows = 0
        num_output_rows = 0
        num_exceptions = 0
        for line in tqdm(fin):
            num_input_rows += 1
            try:
                res = github_pipeline(line)
                num_output_rows += len(res)
            except:
                num_exceptions += 1
        row['num_output_rows'] = num_output_rows
        row['num_input_rows'] = num_input_rows
        row['num_exceptions'] = num_exceptions
        print(row)
        rows.append(row)

48899it [00:13, 3600.77it/s]


{'path': 's3://tuplex-public/data/github_daily/2011-10-15.json', 'num_output_rows': 1418, 'num_input_rows': 48899, 'num_exceptions': 1284}


165692it [00:32, 5068.03it/s] 


{'path': 's3://tuplex-public/data/github_daily/2012-10-15.json', 'num_output_rows': 5907, 'num_input_rows': 165692, 'num_exceptions': 8139}


296456it [00:48, 6168.34it/s]


...

{'path': 's3://tuplex-public/data/github_daily/2019-10-15.json', 'num_output_rows': 61337, 'num_input_rows': 2134789, 'num_exceptions': 0}


2963694it [27:28, 1797.37it/s]


{'path': 's3://tuplex-public/data/github_daily/2020-10-15.json', 'num_output_rows': 62354, 'num_input_rows': 2963694, 'num_exceptions': 0}


1078139it [05:53, 3053.37it/s]

{'path': 's3://tuplex-public/data/github_daily/2021-10-15.json', 'num_output_rows': 21313, 'num_input_rows': 1078139, 'num_exceptions': 0}


In [26]:
import pandas as pd

In [27]:
df = pd.DataFrame(rows)

In [28]:
df

,path,num_output_rows,num_input_rows,num_exceptions
0,s3://tuplex-public/data/github_daily/2011-10-1...,1418,48899,1284
1,s3://tuplex-public/data/github_daily/2012-10-1...,5907,165692,8139
2,s3://tuplex-public/data/github_daily/2013-10-1...,10628,296456,10725
3,s3://tuplex-public/data/github_daily/2014-10-1...,16014,476393,637
4,s3://tuplex-public/data/github_daily/2015-10-1...,25301,737714,0
5,s3://tuplex-public/data/github_daily/2016-10-1...,19412,678134,0
6,s3://tuplex-public/data/github_daily/2017-10-1...,26107,910100,0
7,s3://tuplex-public/data/github_daily/2018-10-1...,44404,1522655,0
8,s3://tuplex-public/data/github_daily/2019-10-1...,61337,2134789,0
9,s3://tuplex-public/data/github_daily/2020-10-1...,62354,2963694,0


In [38]:
df['num_output_rows'].sum()

294195

In [39]:
df['num_input_rows'].sum()

11012665

In [40]:
df['num_exceptions'].sum()

20785

In [29]:
df.to_csv('reference.csv', index=None)

In [33]:
1522655 - (787385 + 490768 + 244446)

56

1522655

In [34]:
56 / 19

2.9473684210526314

In [36]:
df['path'].values

array(['s3://tuplex-public/data/github_daily/2011-10-15.json',
       's3://tuplex-public/data/github_daily/2012-10-15.json',
       's3://tuplex-public/data/github_daily/2013-10-15.json',
       's3://tuplex-public/data/github_daily/2014-10-15.json',
       's3://tuplex-public/data/github_daily/2015-10-15.json',
       's3://tuplex-public/data/github_daily/2016-10-15.json',
       's3://tuplex-public/data/github_daily/2017-10-15.json',
       's3://tuplex-public/data/github_daily/2018-10-15.json',
       's3://tuplex-public/data/github_daily/2019-10-15.json',
       's3://tuplex-public/data/github_daily/2020-10-15.json',
       's3://tuplex-public/data/github_daily/2021-10-15.json'],
      dtype=object)